<a href="https://colab.research.google.com/github/diegogonzalezc/compiled_inventory/blob/main/Cuantificacion_invenarios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Acceso al sistema de archivos
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%pwd

'/content/drive/My Drive/Inventarios '

In [ ]:
%cd '/content/drive/MyDrive/Inventarios '

/content/drive/MyDrive/Inventarios 


In [ ]:
%ls

 Cuantificacion_invenarios.ipynb  'Valorización de inventarios 2022-05.xlsx'


In [ ]:
archivo_inventarios='Valorización de inventarios 2022-05.xlsx'
#Acá se debe copiar y pegar el nombre del archivo qeu contiene toda la informacion entre comillas ''

In [ ]:
excel=pd.ExcelFile(archivo_inventarios) # se crea una lista con los nombres de todas las hojas
hojas_excel=excel.sheet_names
hojas_excel

['Sheet1',
 'balance',
 'almacen',
 'PISOS ALPHA',
 'LIVIN HAUS',
 'SANTA ANA',
 'HADA MTO ANUAL',
 'OFICINA',
 'P15 CALIMA BTA',
 'P16-2021 SN BERNARDO',
 'HOSP STA SSOFIA',
 'P042 TORRES GUARIN',
 'P019 U DE CALDAS 7A',
 'H FABIO TABLAZO',
 'TINY HOUSE',
 'K TRONIX',
 'ROGELIO SALMONA B',
 'P-032-2021 PEOPLE T SIES',
 'ANIBAL FRANCO',
 'P060-2021 U DE CALDAS',
 'P057-2021 ERT YUMBO',
 'GTI BTA AUDIFARMA',
 'P056-2021 VHZ palacio e2']

In [ ]:
base_datos = pd.read_excel(archivo_inventarios,sheet_name='Sheet1',header=7,skipfooter=1) # se crea el dataframe de base de datos de todos los datos de todas las bodegas
base_datos.dropna(subset=['Grupo inventario'],inplace=True) #se eliminan los valoras innecesarios
pd.set_option('display.float_format', lambda x: f'{x:,.3f}') # se cambia el formato para que muestre los numeros en millones
base_datos['Valor unitario']=base_datos['Valor total']/base_datos['Saldo cantidades']# creación de los valores unitarios
base_datos.set_index('Código producto',inplace=True)
base_datos

,Nombre producto,Referencia fábrica,Grupo inventario,Saldo cantidades,Valor total,Valor unitario
Código producto,,,,,,
3001,CAMARA IP PANASONIC 15BL-C14,NaN,EQUIPOS DE COMPUTO,1.000,"175,000.000","175,000.000"
3002,CAMARA PANASONIC WV NM100,NaN,EQUIPOS DE COMPUTO,1.000,"860,000.000","860,000.000"
3004,CAMARA TIPO BALA EXTERIOR,NaN,EQUIPOS DE COMPUTO,1.000,"689,457.000","689,457.000"
3005,CAMARA TIPO DOMO,NaN,EQUIPOS DE COMPUTO,5.000,"812,679.530","162,535.906"
3007,CITOFONO SPRINT BT334202,NaN,EQUIPOS DE COMPUTO,12.000,"686,089.000","57,174.083"
...,...,...,...,...,...,...
5968,PLACA CIEGA PARA BANDEJA DE FIBRA OPTICA,NaN,REDES Y CABLEADO,1.000,"23,403.000","23,403.000"
5969,PLUG BINDADO 6A LEVITON,NaN,REDES Y CABLEADO,110.000,"3,190,000.000","29,000.000"
5970,CANALETA METALICA 10X4 NEGRA,NaN,REDES Y CABLEADO,110.000,"7,088,880.000","64,444.364"


In [ ]:
base_datos_comp=base_datos[['Valor unitario','Grupo inventario']] # se crea una base de datos solo con lo que me interesa comparar
base_datos_comp


,Valor unitario,Grupo inventario
Código producto,,
3001,"175,000.000",EQUIPOS DE COMPUTO
3002,"860,000.000",EQUIPOS DE COMPUTO
3004,"689,457.000",EQUIPOS DE COMPUTO
3005,"162,535.906",EQUIPOS DE COMPUTO
3007,"57,174.083",EQUIPOS DE COMPUTO
...,...,...
5968,"23,403.000",REDES Y CABLEADO
5969,"29,000.000",REDES Y CABLEADO
5970,"64,444.364",REDES Y CABLEADO


In [ ]:
hojas_excel
hojas_excel = [e for e in hojas_excel if e not in ('Sheet1','balance')] # se eliminan sheet 1 y balance de las ojas para no ser analizadas

df_list=[]
count=0
for i in hojas_excel:# este for loop genera un dataframe por cada una de las hojas que hay en el archivo
  i = pd.read_excel(archivo_inventarios,sheet_name=i,header=7,skipfooter=1)
  i.dropna(subset=['Nombre producto'],inplace=True)
  i.pop('Referencia de fábrica')
  i.set_index('Código producto',inplace=True)
  i=pd.merge(i,base_datos_comp,left_index=True, right_index=True)
  i['Valor total']=i['Saldo cantidades']*i['Valor unitario']
  i['Bodega']=hojas_excel[count]
  count+=1
  df_list.append(i)
  
  

In [ ]:
count1=0
for i in df_list:
  if count1 ==0:
    base_datos_bodegas=i
  else:
    base_datos_bodegas=pd.concat([base_datos_bodegas, i])
  count1+=1


In [ ]:
base_datos_bodegas

,Nombre producto,Saldo cantidades,Valor unitario,Grupo inventario,Valor total,Bodega
Código producto,,,,,,
1001,BATERIA CSB 17AH-12VDC,2.000,"81,150.500",BATERIAS UPS Y EQ ELECTRICOS,"162,301.000",almacen
1002,BATERIA CSB 12VDC-5AH,1.000,"48,428.573",BATERIAS UPS Y EQ ELECTRICOS,"48,428.573",almacen
1003,BATERIA CSB 12VDC-7AH,2.000,"63,357.363",BATERIAS UPS Y EQ ELECTRICOS,"126,714.725",almacen
1004,BATERIA 6VDC-4AH,8.000,"15,618.200",BATERIAS UPS Y EQ ELECTRICOS,"124,945.600",almacen
1005,BATERIA FULL BATERY 12VDC-7AH,1.000,"37,297.077",BATERIAS UPS Y EQ ELECTRICOS,"37,297.077",almacen
...,...,...,...,...,...,...
5910,SENSOR LEVITON PROVOL BLUETOOTH DIMERI,20.000,"788,141.000",REDES Y CABLEADO,"15,762,820.000",P056-2021 VHZ palacio e2
5911,INTERRUPTOR LEVITON PLVSW-4W PROVOL 4 BOTON BL...,17.000,"210,625.000",REDES Y CABLEADO,"3,580,625.000",P056-2021 VHZ palacio e2
5917,CAJA GALVANIZADA OCTOGONAL,16.000,"1,600.000",REDES Y CABLEADO,"25,600.000",P056-2021 VHZ palacio e2


In [ ]:
Total = base_datos_bodegas['Valor total'].sum()
Total

2388342242.0600004

In [ ]:
resumen=base_datos_bodegas.groupby('Bodega').agg({'Saldo cantidades':'sum','Valor total':'sum'})

resumen

,Saldo cantidades,Valor total
Bodega,,
ANIBAL FRANCO,473.000,"41,142,595.511"
GTI BTA AUDIFARMA,"9,953.000","30,771,290.505"
H FABIO TABLAZO,"2,448.000","7,122,071.978"
HADA MTO ANUAL,25.000,"185,532.007"
HOSP STA SSOFIA,4.000,"92,926.213"
K TRONIX,1.000,"24,213.028"
LIVIN HAUS,"50,289.000","141,835,939.772"
OFICINA,14.000,"6,065,836.669"
P-032-2021 PEOPLE T SIES,65.000,"4,350,194.996"


In [ ]:
Total = resumen['Valor total'].sum()
Total

2388342242.06

,Saldo cantidades,Valor total
Nombre producto,,
ABRAZADERA DE 2 PULGADAS,353.000,"495,734.980"
ABRAZADERA 3/4,"2,008.000","533,338.780"
ABRAZADERA COLGANTE TIPO PERA DE 1 1/2,69.000,"84,800.000"
ABRAZADERA DE 1 1/2,171.000,"107,606.300"
"ABRAZADERA DE 1 1/4""",14.000,"11,100.000"
...,...,...
VELCRO SIEMON PAQUETE 250 UND,9.000,"2,569,895.000"
VENTILADOR CON FILTRO,2.000,"330,176.000"
VENTILADOR PARA GABINETE DE PARED,1.000,"38,194.500"
